# Opik Optimizer (beta)

Welcome to the Opik Opitimizer beta program! In this notebook we'll walk through a basic examples of a number of an optimizer algorithm.

## Setup

First, let's install the needed Python packages.

We'll of course need `opik`:

In [1]:
%pip install opik --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.3/149.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.7/458.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 114.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.0 MB/s eta 0:00:00


Next, we'll install the beta version of the `opik-optimizer`:

(There may be some package conflicts with torch and nvidia libraries, but that is fine).

In [2]:
%pip install git+https://github.com/comet-ml/opik#subdirectory=sdks/opik_optimizer --upgrade --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 83.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 118.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.0/259.0 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 124.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.6/460.6 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/5

(Ignore any errors above)


[Comet](https://www.comet.com/site?from=llm&utm_source=opik) provides a hosted version of the Opik platform, [simply create an account](https://www.comet.com/signup?from=llm&utm_source=opik&utm_medium=colab&utm_content=langchain&utm_campaign=opik) and grab your API Key.

> You can also run the Opik platform locally, see the [installation guide](https://www.comet.com/docs/opik/self-host/overview/?from=llm&utm_source=opik) for more information.

You can use your own workspace.

In [3]:
import opik

# Configure Opik
opik.configure()

OPIK: Your Opik API key is available in your account settings, can be found at https://www.comet.com/api/my/settings/ for Opik cloud


Please enter your Opik API key:··········
Do you want to use "dsblank" workspace? (Y/n)y


OPIK: Configuration saved to file: /root/.opik.config


For this example, we'll use OpenAI models, so we need to set our OpenAI API key:

In [4]:
import os
import getpass
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


To optimize any prompt, we'll need:

1. A starting prompt
2. A metric
3. A dataset

For this initial test, we'll start with a portion of the HotpotQA dataset.

HotpotQA is a question/answering dataset featuring natural questions, with correct answers. It was collected by a team of NLP researchers at Carnegie Mellon University, Stanford University, and Université de Montréal.

Let's take a look at the demo dataset "hotpot-300".

First, we get or create it. This will add it to our default Opik project:

In [5]:
from opik_optimizer.demo import get_or_create_dataset

opik_dataset = get_or_create_dataset("hotpot-300")

Let's look at a specific row from the dataset:

In [6]:
rows = opik_dataset.get_items()
rows[0]

{'id': '0195d400-517b-7f17-b746-ff3d084463ac',
 'question': 'Were both drinks, the Smoking Bishop and the Mickey Slim, popular in different countries?',
 'answer': 'yes'}

We see that each item has a "question" and "answer". Some of the answers are short and direct, and some of them are more complicated:

In [7]:
rows[2]

{'id': '0195d400-5179-7259-8104-1f0a55a13ac2',
 'question': 'Woody Wuthrie wrote the song "Do Re Mi" which was about migrants and their experiences when they arrive in California and are greeted with severe storms that came in waves in what years?',
 'answer': '1934, 1936, and 1939–1940'}

As you can see, the answers can be a little messy. We'll need a good metric to able to determine whether a prompt is good or not. But let's start simple.

We'll use the `LevenshteinRatio` metric from Opik.

It works like this:

In [8]:
from opik.evaluation.metrics import LevenshteinRatio

metric = LevenshteinRatio()

metric.score("Hello", "Hello")

OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=0196213d-a988-737a-88e8-f069b6408310&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


ScoreResult(name='levenshtein_ratio_metric', value=1.0, reason=None, metadata=None, scoring_failed=False)

Here we can see that value is 1.0. That meens that there is no difference between "Hello" and "Hello". That is as close as you can get!

As the difference between the two strings get further away, the value gets smaller, approaching 0.0:

In [9]:
metric.score("Hello", "Hello!")

ScoreResult(name='levenshtein_ratio_metric', value=0.9090909090909091, reason=None, metadata=None, scoring_failed=False)

In [10]:
metric.score("one", "six")

ScoreResult(name='levenshtein_ratio_metric', value=0.0, reason=None, metadata=None, scoring_failed=False)

As you can see, slightly different have values close to 1, and no letters in common give a zero.

To find out more about the metric, see [Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance).

Ok, we have a dataset and a metric, now we are ready to construct an optimizer.

We can use an OpenAI model name, or more generally, a LiteLLM model name. Just make sure you have your model API key set as we did above.

We'll start with the `FewShotOptimizer`:

In [11]:
from opik_optimizer import FewShotOptimizer

optimizer = FewShotOptimizer(
    model="openai/gpt-4o-mini",
    temperature=0.1,
    max_tokens=5000,
)

Now, we need a prompt to optimize. Given the examples above, let's try something like:

In [13]:
initial_prompt = "Provide an answer to the question"

In [15]:
score = optimizer.evaluate_prompt(
    dataset=opik_dataset,
    metric=metric,
    prompt=initial_prompt,
    # Algorithm-specific kwargs:
    input_key="question",
    output_key="answer",
)
score

100%|██████████| 10/10 [00:00<00:00, 1514.35it/s]


0.11992387410132233

The score 0.11 is pretty low. Let's see if we can optimize it!

The FewShotOptimizer can, fairly quickly, create better prompts.

Let's try it out. It takes exactly the same parameters as evaluate_prompt(), but will run for a minute or so.

In [16]:
results = optimizer.optimize_prompt(
    dataset=opik_dataset,
    metric=metric,
    prompt="Answer the question with a short, 1 to 5 word phrase",
    # Algorithm-specific kwargs:
    input_key="question",
    output_key="answer",
)

Step 1: Bootstrap few-shot examples...
Bootstrapping set 1/5
Bootstrapping set 2/5
Bootstrapping set 3/5


  7%|▋         | 4/60 [00:03<00:54,  1.02it/s]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/5


  3%|▎         | 2/60 [00:07<03:32,  3.66s/it]


Bootstrapped 2 full traces after 2 examples for up to 1 rounds, amounting to 2 attempts.
Bootstrapping set 5/5


  7%|▋         | 4/60 [00:03<00:46,  1.21it/s]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Step 2: Propose instruction candidates...


100%|██████████| 10/10 [00:06<00:00,  1.55it/s]


In [17]:
print(results)


Optimization Results:
    Best prompt: Provide a brief answer (1-5 words) to the given trivia question.
    Best score: 0.5740



Although there is some randomness, you probably got a better prompt. My result is about 5 times better.

This is just the initial framework for optimizers for Opik!

Please see the [Opik Optimizer repo](https://github.com/comet-ml/opik/tree/main/sdks/opik_optimizer) for additional examples using different algorithms.